In [63]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.functions import*
from pyspark.sql.types import*
spark = SparkSession.builder.appName('practise').getOrCreate()
import os 
import pandas as pd

In [64]:
spark

In [ ]:
df=spark.read.csv(path='nuga_bank_transactions (1).csv',header=True,inferSchema=True)
df.printSchema()

In [ ]:
for column in df.columns:
    print(f'{column} null count :{df.where(df[column].isNull()).count()}')

In [67]:
df = df.na.fill({
    'Customer_Name': 'unknown',
    'Customer_Address': 'unknown',
    'Customer_City': 'unknown',
    'Customer_State': 'unknown',
    'Company': 'unknown',
    'Job_Title': 'unknown',
    'Email': 'unknown',
    'Phone_Number': 0.0,
    'Random_Number': 0.0,
    'Credit_Card_Number': 0.0,
    'Category': 'unknown',
    'IBAN': 'unknown',
    'Currency_Code': 0.0,
    'Group': 'unknown',
    'Is_Active': 'unknown',
    'Last_Updated': 'unknown',
    'Description': 'unknown',
    'Gender': 'unknown',
    'Marital_Status': 'unknown',
    'Last_Updated':0.0,
    'Customer_Country':'unknown'
})

In [ ]:
for column in df.columns:
    print(f'{column} null count :{df.where(df[column].isNull()).count()}')

In [ ]:
#chaeck the column name for data modeling
df.columns

In [69]:
# customer table from the data modeling
customer = df.select(['Customer_Name', 'Customer_Address', 'Customer_City', 'Customer_State', 'Customer_Country']).distinct()
customer = customer.withColumn('Customerid', monotonically_increasing_id()).distinct()
customer=customer.select(['Customerid','Customer_Name', 'Customer_Address', 'Customer_City', 'Customer_State', 'Customer_Country']).distinct()
customer.show()

+----------+----------------+--------------------+------------------+--------------+--------------------+
|Customerid|   Customer_Name|    Customer_Address|     Customer_City|Customer_State|    Customer_Country|
+----------+----------------+--------------------+------------------+--------------+--------------------+
|         0|   Latoya Robles|  6874 Jones Centers|      Adamschester|      Delaware| Antigua and Barbuda|
|         1|     Jamie Dixon|0146 Veronica Mou...|         Jonesland|      Delaware|        Saint Martin|
|         2|     Vicki James|7987 Carr Bridge ...|           unknown| West Virginia|         Puerto Rico|
|         3|Jennifer Kennedy|     388 Susan Forks|        East Jason|          Ohio|United States of ...|
|         4|     Susan Green|40609 Amber Junct...|          Markfort|      Colorado|       Liechtenstein|
|         5| Elizabeth Hicks|             unknown|           unknown|         Maine|          Micronesia|
|         6|    Emily Foster|24260 Gilbert Bur

In [72]:
Transaction = df.select(['Transaction_Date','Transaction_Type','Amount']).distinct()
Transaction =Transaction.withColumn('Transactionid',monotonically_increasing_id())
Transaction =Transaction .select(['Transactionid','Transaction_Date','Transaction_Type','Amount'])
Transaction .show()

+-------------+--------------------+----------------+------+
|Transactionid|    Transaction_Date|Transaction_Type|Amount|
+-------------+--------------------+----------------+------+
|            0|2024-04-22 19:15:...|      Withdrawal|163.92|
|            1|2024-02-27 16:38:...|        Transfer|270.15|
|            2|2024-02-29 07:07:...|      Withdrawal|320.56|
|            3|2024-01-17 11:30:...|         Deposit|364.18|
|            4|2024-04-23 08:58:...|        Transfer|997.73|
|            5|2024-02-01 17:06:...|        Transfer|353.25|
|            6|2024-04-24 19:56:...|         Deposit|337.15|
|            7|2024-02-13 06:17:...|      Withdrawal|251.91|
|            8|2024-02-18 00:26:...|         Deposit|281.84|
|            9|2024-03-22 09:55:...|        Transfer| 23.87|
|           10|2024-03-16 00:18:...|        Transfer|978.53|
|           11|2024-02-21 21:58:...|      Withdrawal|423.97|
|           12|2024-04-01 07:04:...|      Withdrawal|270.33|
|           13|2024-03-2

In [73]:
Employee=df.select(['Company','Category','Job_Title','Gender','Marital_Status']).distinct()
Employee=Employee.withColumn('Employeeid',monotonically_increasing_id()).distinct()
Employee=Employee.select(['Employeeid','Company','Category','Job_Title','Gender','Marital_Status'])
Employee.show()

+----------+--------------------+--------+--------------------+-------+--------------+
|Employeeid|             Company|Category|           Job_Title| Gender|Marital_Status|
+----------+--------------------+--------+--------------------+-------+--------------+
|         0|           Adams Inc|       B|       Ranger/warden|  Other|      Divorced|
|         1|        Mann-Ramirez|       A|Production design...| Female|        Single|
|         2|          Vega Group|       C|Scientist, clinic...|  Other|      Divorced|
|         3|   Valenzuela-Wilson|       D|Forest/woodland m...|  Other|        Single|
|         4|    Johnson and Sons|       D|Fitness centre ma...|   Male|        Single|
|         5|Novak, Baker and Lee|       C|         Media buyer|  Other|       unknown|
|         6|             unknown|       A|   Buyer, industrial|  Other|      Divorced|
|         7|             unknown|       A|      Data scientist| Female|        Single|
|         8|             unknown|       B|H

In [75]:
FactTable = df.join(customer,['Customer_Name', 'Customer_Address', 'Customer_City', 'Customer_State', 'Customer_Country'],'left')\
            .join(Transaction,['Transaction_Date','Transaction_Type','Amount'], 'left')\
            .join(Employee,['Company','Category','Job_Title','Gender','Marital_Status'],'left')
          

In [ ]:
FactTable=FactTable.withColumn('Factid',monotonically_increasing_id()).distinct()
FactTable=FactTable.select(['Factid','Transactionid','Employeeid','Customerid','Credit_Card_Number','IBAN','Currency_Code','Random_Number','Last_Updated','Description','Is_Active','Group']).distinct()
FactTable.show()

In [ ]:
FactTable.toPandas().to_csv('FactTable.csv', index=False)

In [ ]:
Employee.toPandas().to_csv('Employee.csv', index=False)


In [88]:
Transaction.toPandas().to_csv('Transaction.csv', index=False)



In [ ]:
customer.toPandas().to_csv('customer.csv', index=False)


In [ ]:
import psycopg2
from sqlalchemy import create_engine

conn = None
try:
    print('Connecting to the database...')
    conn = psycopg2.connect(
        host='localhost',
        user='postgres',
        dbname='bluewave',
        password='1',
        port=5432
    )
    
    print('Connected to the database!')

    cur = conn.cursor()
    cur.execute('SELECT version()')
    result = cur.fetchone()
    print(result)

except (Exception, psycopg2.DatabaseError) as error:
    print('Error:', error)
finally:
    if conn is not None:
        conn.close()
        print('Database connection closed.')


